In [17]:
import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.models import vgg19
from PIL import Image

import argparse
import os
import numpy as np
import math
import itertools
import sys

import torchvision.transforms as transforms
from torchvision.utils import save_image, make_grid

from torch.utils.data import DataLoader
from torch.autograd import Variable

import torch.nn as nn
import torch.nn.functional as F
import torch

In [6]:
# Use cuda
cuda = torch.cuda.is_available()
# torch.cuda.set_device(2)

In [26]:
# Define vgg19 feature extractor
class FeatureExtractor(nn.Module):
    def __init__(self):
        super(FeatureExtractor, self).__init__()
        vgg19_model = vgg19(pretrained=True)
        self.feature_extractor = nn.Sequential(*list(vgg19_model.features.children())[:2])

    def forward(self, img):
        return self.feature_extractor(img)

In [3]:
feature_extractor = FeatureExtractor()
# Set feature extractor to inference mode
feature_extractor.eval()
if cuda:
    feature_extractor = feature_extractor.cuda()


In [15]:
# Load a image and transfer it to torch.tensor format
img_transform = transforms.Compose(
            [
                transforms.ToTensor(),
                transforms.Normalize(0.5, 0.5),
            ]
        )

img = Image.open('/home/ubuntu/super_resolution/PyTorch-GAN/implementations/srgan/50799.png')
img_1 = img_transform(img)

In [ ]:
# Transfer the image to torch.cuda.tensor format to use cuda (if cuda is not used, then delete related code where cuda is used)
Tensor = torch.cuda.FloatTensor if cuda else torch.Tensor
img_1 = img_1.type(Tensor)

In [28]:
# Generate features
gen_features = feature_extractor(img_1)
gen_features

tensor([[[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         ...,
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000]],

        [[1.2252, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [2.9541, 0.0000, 1.4157,  ..., 4.5807, 2.4732, 0.0000],
         [2.9541, 0.0000, 0.0000,  ..., 4.9901, 4.0407, 0.0000],
         ...,
         [2.9541, 0.0000, 0.0000,  ..., 0.4021, 1.7186, 0.0000],
         [2.9541, 0.0000, 0.0000,  ..., 0.1869, 0.8947, 0.0000],
         [3.7617, 2.2921, 2.2921,  ..., 2.2921, 2.2921, 0.2020]],

        [[0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.0000],
         [0.0000, 0.0000, 0.0000,  ..., 0.0000, 0.0000, 0.